In [1]:
!pip install ipywidgets
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install --upgrade jupyter_client

You should consider upgrading via the 'pip install --upgrade pip' command.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Requirement already up-to-date: jupyter_client in /Users/zzy824/anaconda2/envs/py36/lib/python3.6/site-packages (6.1.3)


You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
batch_size = 32
lr = 0.01
epoch_number = 100
hidden_nodes = 64
dataset_name = 'cifar10' # cifar10; fashion_mnist
## cifar10 (You may need to add or modify the parameters)
color_channels = 3
image_cifar10_width = 32 # width of the cifar10 image
image_cifar10_height = 32 # height of the cifar10 image
class_number_cifar10 = 10 # number of classes in CIFAR10
dropout = 0.0
## fashion mnist (You may need to add or modify the parameters)
image_fashion_mnist_width = 28 # width of the fashion mnist image
image_fashion_mnist_height = 28 # height of the fashion mnist image
class_number_fashion_mnist = 10 # number of classes in fashion mnist
visual_flag = False # To determine whether to visualize the weights of neural network or not
log_interval = 1 # interval to print results to screen
save_weight_interval = 5 # interval to save network visualization figure to disk
data_path = './data'
seed = 824

/Users/zzy824/Google Drive/UTA-homeworks/2020_Spring/DataMining/CSE5334_DataMining/assignment3/q2


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from ipywidgets import IntProgress
import matplotlib as mpl
import matplotlib.pyplot as plt
torch.manual_seed(seed)
mpl.use('Agg')


plt.style.use('ggplot')

CIFAR10_classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [29]:
# Helper functions
def data_subset(raw_data_list, class_list):
    '''
    This function is used to select part of the dataset. The selected classes are listed in the input argument --- "class_list".
    '''
    subset_data_list = []
    for i in range(len(raw_data_list)):
        for j in range(len(class_list)):
            if raw_data_list[i][1] == class_list[j]:
                each_data = []
                each_data.append(raw_data_list[i][0])
                each_data.append(j)
                subset_data_list.append(each_data)
            else:
                continue
    print("subset_data_list len: ", len(subset_data_list))
    return subset_data_list


def data_loader(dataset_name):
    '''
    CIFAR10:       [0 airplane,    1 automobile, 2 bird,     3 cat,   4 deer, 5 dog,    6 frog,  7 horse,   8 ship, 9 truck]
    Fashion MNIST: [0 T-shirt/top, 1 Trouser,    2 Pullover, 3 Dress, 4 Coat, 5 Sandal, 6 Shirt, 7 Sneaker, 8 Bag,  9 Ankle boot]

    train_data, test_data: type -- list; You need to transform them into numpy if you want to use them for logistic regression.
    '''
    cifar10_transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    fashion_transform = transforms.Compose(
        [transforms.ToTensor(), 
         transforms.Normalize([0.5], [0.5])])

    ## The index of the classes selected, for example: [0, 1]. If none indices are selected, the whole dataset will be used.
    cifar10_class_list = []        ## You may need to change the index number if you want to select different classes.
    fashion_mnist_class_list = [0, 1]  ## You may need to change the index number if you want to select different classes.

    ## generate data loader for CIFAR10 and Fashion MNIST
    if dataset_name == "cifar10":
        train_data_raw = torchvision.datasets.CIFAR10(root='./data', train=True, download=True,
                                                      transform=cifar10_transform)
        test_data_raw = torchvision.datasets.CIFAR10(root='./data', train=False, download=True,
                                                     transform=cifar10_transform)

        if len(cifar10_class_list) > 0:
            train_data = data_subset(train_data_raw, cifar10_class_list)
            test_data = data_subset(test_data_raw, cifar10_class_list)
        else:
            train_data = train_data_raw
            test_data = test_data_raw
    elif dataset_name == "fashion_mnist":
        train_data_raw = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True,
                                                       transform=fashion_transform)
        test_data_raw = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True,
                                                      transform=fashion_transform)
        if len(fashion_mnist_class_list) > 0:
            train_data = data_subset(train_data_raw, fashion_mnist_class_list)
            test_data = data_subset(test_data_raw, fashion_mnist_class_list)
        else:
            train_data = train_data_raw
            test_data = test_data_raw

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

    print("Working on {} dataset ... ".format(dataset_name))
    print("len of train_loader dataset: ", len(train_loader.dataset))
    print("len of test_loader dataset: ", len(test_loader.dataset))

    return train_loader, test_loader, train_data, test_data

# save the visualization of weights into a file
def save_fig(inner_matrix, inner_file_name):
    inner_fig = plt.figure()
    for i in range(inner_matrix.shape[0]):
        ax = inner_fig.add_subplot(1, inner_matrix.shape[0], i + 1)
        ax.imshow(inner_matrix[i].reshape(image_fashion_mnist_width, image_fashion_mnist_height))
        plt.xticks([])
        plt.yticks([])
    inner_fig.savefig(inner_file_name)
    plt.close(inner_fig)

In [97]:
import torch
import torch.nn.functional as F

class Net(torch.nn.Module):
    def __init__(self, one_more_hid=False, freestyle=False, vanilla=False, logistic=False):
        super(Net, self).__init__()
        self.vanilla = vanilla
        self.logistic = logistic
        self.dropout = dropout
        self.fc1 = torch.nn.Linear(image_cifar10_width*image_cifar10_height*color_channels, hidden_nodes)
        self.fc1_fashion = torch.nn.Linear(image_fashion_mnist_width*image_fashion_mnist_height, hidden_nodes)
        self.fc2 = None
        self.fc3 = None
        self.bn1 = None
        self.bn2 = None
        self.bn3 = None
        self.non_hidden = torch.nn.Linear(image_fashion_mnist_width*image_fashion_mnist_height, class_number_fashion_mnist)
        if one_more_hid:
            self.fc2 = torch.nn.Linear(hidden_nodes, hidden_nodes)
        if freestyle:
            self.bn1 = torch.nn.BatchNorm1d(num_features=64)
            self.fc2 = torch.nn.Linear(hidden_nodes, 128)
            self.bn2 = torch.nn.BatchNorm1d(num_features=128)
            self.fc3 = torch.nn.Linear(128, 64)
            self.bn3 = torch.nn.BatchNorm1d(num_features=64)
        if logistic:
            self.logi_fc = torch.nn.Linear(image_fashion_mnist_width*image_fashion_mnist_height, 1)
        self.out_cifar10 = torch.nn.Linear(hidden_nodes, class_number_cifar10)
        self.out_fashion_mnist = torch.nn.Linear(hidden_nodes, class_number_fashion_mnist)

    def forward(self, x):
        if dataset_name == "cifar10":
            x = x.view(-1, image_cifar10_width*image_cifar10_height*color_channels)
            x = self.fc1(x)
            if self.bn1:
                x = self.bn1(x)
            x = torch.nn.functional.relu(x)
            x = torch.nn.Dropout(self.dropout)(x)
            if self.fc2:
                x = self.fc2(x)
                if self.bn2:
                    x = self.bn2(x)
                x = torch.nn.functional.relu(x)
                x = torch.nn.Dropout(self.dropout)(x)
            if self.fc3:
                x = self.fc3(x)
                if self.bn3:
                    x = self.bn3(x)
                x = torch.nn.functional.relu(x)
                x = torch.nn.Dropout(self.dropout)(x)
            return self.out_cifar10(x)
        elif dataset_name == "fashion_mnist":
            if self.logistic:
                x = x.view(-1, image_fashion_mnist_width*image_fashion_mnist_height)
                return torch.nn.functional.sigmoid(self.logi_fc(x))
            elif not self.vanilla:
                x = x.view(-1, image_fashion_mnist_width*image_fashion_mnist_height)
                x = self.fc1_fashion(x)
                if self.bn1:
                    x = self.bn1(x)
                x = torch.nn.functional.relu(x)
                x = torch.nn.Dropout(self.dropout)(x)
                if self.fc2:
                    x = self.fc2(x)
                    if self.bn2:
                        x = self.bn2(x)
                    x = torch.nn.functional.relu(x)
                    x = torch.nn.Dropout(self.dropout)(x)
                if self.fc3:
                    x = self.fc3(x)
                    if self.bn3:
                        x = self.bn3(x)
                    x = torch.nn.functional.relu(x)
                    x = torch.nn.Dropout(self.dropout)(x)
                return self.out_fashion_mnist(x)
            else:
                x = x.view(-1, image_fashion_mnist_width*image_fashion_mnist_height)
                x = self.non_hidden(x)
                return x, self.non_hidden.weight.data

In [145]:
# Train test
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import os
import numpy as np

## train the model
def train(model, train_loader, optimizer, epoch_index, logit=False):
    model.train()

    correct = 0
    epoch_train_loss = 0.0
    epoch_labels = None
    epoch_preds = None

    ## use a weight matrix to store the weight of the network which will be used for visualization
    weight_matrix = np.zeros((image_fashion_mnist_width, image_fashion_mnist_height))
    if logit:
        loss_func = torch.nn.BCELoss()

    for batch_idx, (data, target) in enumerate(train_loader):

        ## preparing the data fed into the neural network
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()

        ## obtain the output of the network
        if dataset_name == "cifar10":
            output = model(data)
        elif dataset_name == "fashion_mnist":
            # if viusalize the weights of neural network
            if visual_flag:
                output, weight_ret = model(data)
                weight_matrix = weight_ret.detach().numpy()
            # not viusalize the weights of neural network
            else:
                output = model(data)
        # calculate the predication
        if not logit:
            train_pred = torch.argmax(F.softmax(output, dim=1), dim=1).view(-1, )
        else:
            train_pred = (output>0.5).float().squeeze()
        # count the correct prediction
        correct += train_pred.eq(target.data).sum()

        # The cross entropy loss is calculated, softmax function is embedded in F.cross_entropy() function
        if not logit:
            loss = F.cross_entropy(output, target)
        else:
            target_float = target.float()
            loss = loss_func(output, target_float)
        epoch_train_loss += loss.item()

        # loss is used for back-propagation (BP) in MLP
        loss.backward()
        optimizer.step()
        # record
        if epoch_index==epoch_number-1:
            if epoch_labels == None:
                epoch_labels = target
            else:
                epoch_labels = torch.cat((epoch_labels, target))
            if epoch_preds == None:
                epoch_preds = train_pred
            else:
                epoch_preds = torch.cat((epoch_preds, train_pred))
            
            
    epoch_train_accuracy = 100. * correct / len(train_loader.dataset)
    epoch_loss_mean = epoch_train_loss / len(train_loader)

    if epoch_index % log_interval == 0:
        print(correct, len(train_loader.dataset))
        print('Train Epoch: {}\tLoss: {:.6f} Accuracy: {:.2f}%'.format(epoch_index, epoch_loss_mean,
                                                                       epoch_train_accuracy))

    # visualize the weights of neural network for Fashion MNIST dataset
    if dataset_name == "fashion_mnist" and visual_flag and epoch_index % save_weight_interval == 0:
        os.makedirs('./output2.6', exist_ok=True)
        save_fig(weight_matrix, os.path.join('./output2.6', "network_weights_" + str(epoch_index) + ".pdf"))

    return epoch_loss_mean, epoch_train_accuracy, epoch_labels, epoch_preds


## test the model
def test(model, test_loader, epoch_index, logit=False):
    model.eval()
    epoch_test_loss, correct = 0, 0
    epoch_labels, epoch_preds = None, None
    if logit:
        loss_func = torch.nn.BCELoss()        

    # testing does not need the gradient
    with torch.no_grad():
        for data, target in test_loader:
            data, target = Variable(data), Variable(target)

            if dataset_name == "cifar10":
                output = model(data)
            elif dataset_name == "fashion_mnist":
                if visual_flag:
                    output, _ = model(data)
                else:
                    output = model(data)

            # the process is the same with training part
            if not logit:
                epoch_test_loss += F.cross_entropy(output, target).item()
                test_pred = torch.argmax(F.softmax(output, dim=1), dim=1).view(-1, )
            else:
                epoch_test_loss += loss_func(output, target.float()).item()
                test_pred = (output>0.5).float().squeeze()
                
            correct += test_pred.eq(target.data).cpu().sum()
            
            if epoch_index==epoch_number-1:
                if epoch_labels == None:
                    epoch_labels = target
                else:
                    epoch_labels = torch.cat((epoch_labels, target))
                if epoch_preds == None:
                    epoch_preds = test_pred
                else:
                    epoch_preds = torch.cat((epoch_preds, test_pred))

        epoch_test_loss_mean = epoch_test_loss / len(test_loader)

    epoch_test_accuracy = 100. * correct / len(test_loader.dataset)

    if epoch_index % log_interval == 0:
        print('\n      Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            epoch_test_loss_mean, correct, len(test_loader.dataset), epoch_test_accuracy))

    return epoch_test_loss_mean, epoch_test_accuracy, epoch_labels, epoch_preds


In [44]:
##### 2.1 vanilla mlp
## data loaders for training and testing;
train_loader, test_loader, train_data, test_data = data_loader(dataset_name)
## load the multi-layered perceptron (MLP) model
mlp_model = Net()

## optimizer used to implement weight update using gradient descent
optimizer = torch.optim.SGD(mlp_model.parameters(), lr=lr)

## train the model
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []

for epoch_idx in range(epoch_number):
    train_loss_ret, train_acc_ret, _, _ = train(mlp_model, train_loader, optimizer, epoch_idx)
    train_loss_list.append(train_loss_ret)
    train_acc_list.append(train_acc_ret)

    test_loss_ret, test_acc_ret, _, _ = test(mlp_model, test_loader, epoch_idx)
    test_loss_list.append(test_loss_ret)
    test_acc_list.append(test_acc_ret)

## plot loss
x_axis = np.arange(epoch_number)
loss_fig = plt.figure()
plt.plot(x_axis, train_loss_list, 'r')
plt.plot(x_axis, test_loss_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(("train", "test"))

os.makedirs('./output2.1', exist_ok=True)
loss_fig.savefig(os.path.join('./output2.1', dataset_name + "_loss.pdf"))
plt.close(loss_fig)
## plot accuracy
acc_fig = plt.figure()
plt.plot(x_axis, train_acc_list, 'r')
plt.plot(x_axis, test_acc_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(("train", "test"))
os.makedirs('./output2.1', exist_ok=True)
acc_fig.savefig(os.path.join('./output2.1', dataset_name + "_accuracy.pdf"))
plt.close(acc_fig)

## save loss and accuracy to the file
loss_acc_np = np.arange(epoch_number).reshape((-1, 1))
loss_acc_np = np.append(loss_acc_np, np.array(train_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(train_acc_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_acc_list).reshape((-1, 1)), axis=1)
os.makedirs('./output2.1', exist_ok=True)
np.savetxt(os.path.join('./output2.1', dataset_name + "_loss_acc.txt"), loss_acc_np)

subset_data_list len:  12000
subset_data_list len:  2000
Working on fashion_mnist dataset ... 
len of train_loader dataset:  12000
len of test_loader dataset:  2000


RuntimeError: stack expects each tensor to be equal size, but got [2, 32] at entry 0 and [32] at entry 1

In [38]:
!pwd
!pip install scikit-plot

/Users/zzy824/Google Drive/UTA-homeworks/2020_Spring/DataMining/CSE5334_DataMining/assignment3/q2
You should consider upgrading via the 'pip install --upgrade pip' command.


In [18]:
##### 2.2

## data loaders for training and testing;
train_loader, test_loader, train_data, test_data = data_loader(dataset_name)
## load the multi-layered perceptron (MLP) model
mlp_model = Net(one_more_hid=True)

## optimizer used to implement weight update using gradient descent
optimizer = torch.optim.SGD(mlp_model.parameters(), lr=lr)

## train the model
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []

for epoch_idx in range(epoch_number):
    train_loss_ret, train_acc_ret, _, _ = train(mlp_model, train_loader, optimizer, epoch_idx)
    train_loss_list.append(train_loss_ret)
    train_acc_list.append(train_acc_ret)

    test_loss_ret, test_acc_ret, _, _ = test(mlp_model, test_loader, epoch_idx)
    test_loss_list.append(test_loss_ret)
    test_acc_list.append(test_acc_ret)

## plot loss
x_axis = np.arange(epoch_number)
loss_fig = plt.figure()
plt.plot(x_axis, train_loss_list, 'r')
plt.plot(x_axis, test_loss_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(("train", "test"))
os.makedirs('./output2.2', exist_ok=True)
loss_fig.savefig(os.path.join('./output2.2', dataset_name + "_loss.pdf"))
plt.close(loss_fig)

## plot accuracy
acc_fig = plt.figure()
plt.plot(x_axis, train_acc_list, 'r')
plt.plot(x_axis, test_acc_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(("train", "test"))
os.makedirs('./output2.2', exist_ok=True)
acc_fig.savefig(os.path.join('./output2.2', dataset_name + "_accuracy.pdf"))
plt.close(acc_fig)

## save loss and accuracy to the file
loss_acc_np = np.arange(epoch_number).reshape((-1, 1))
loss_acc_np = np.append(loss_acc_np, np.array(train_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(train_acc_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_acc_list).reshape((-1, 1)), axis=1)
os.makedirs('./output2.2', exist_ok=True)
np.savetxt(os.path.join('./output2.2', dataset_name + "_loss_acc.txt"), loss_acc_np)

Files already downloaded and verified
Files already downloaded and verified
Working on cifar10 dataset ... 
len of train_loader dataset:  50000
len of test_loader dataset:  10000
Train Epoch: 0	Loss: 1.930804 Accuracy: 31.23%

      Test set: Average loss: 1.7284, Accuracy: 3838/10000 (38%)

Train Epoch: 1	Loss: 1.667905 Accuracy: 40.88%

      Test set: Average loss: 1.6007, Accuracy: 4342/10000 (43%)

Train Epoch: 2	Loss: 1.566226 Accuracy: 44.73%

      Test set: Average loss: 1.5450, Accuracy: 4587/10000 (46%)

Train Epoch: 3	Loss: 1.497523 Accuracy: 47.31%

      Test set: Average loss: 1.4908, Accuracy: 4753/10000 (48%)

Train Epoch: 4	Loss: 1.445183 Accuracy: 49.41%

      Test set: Average loss: 1.4695, Accuracy: 4820/10000 (48%)

Train Epoch: 5	Loss: 1.401459 Accuracy: 50.89%

      Test set: Average loss: 1.4544, Accuracy: 4853/10000 (49%)

Train Epoch: 6	Loss: 1.364503 Accuracy: 51.85%

      Test set: Average loss: 1.4234, Accuracy: 4931/10000 (49%)

Train Epoch: 7	Loss: 1.

Train Epoch: 70	Loss: 0.684165 Accuracy: 75.72%

      Test set: Average loss: 1.8892, Accuracy: 4939/10000 (49%)

Train Epoch: 71	Loss: 0.682148 Accuracy: 75.94%

      Test set: Average loss: 1.9633, Accuracy: 4776/10000 (48%)

Train Epoch: 72	Loss: 0.675708 Accuracy: 76.16%

      Test set: Average loss: 2.0093, Accuracy: 4814/10000 (48%)

Train Epoch: 73	Loss: 0.669380 Accuracy: 76.23%

      Test set: Average loss: 1.9866, Accuracy: 4911/10000 (49%)

Train Epoch: 74	Loss: 0.667090 Accuracy: 76.39%

      Test set: Average loss: 1.9888, Accuracy: 4823/10000 (48%)

Train Epoch: 75	Loss: 0.660155 Accuracy: 76.76%

      Test set: Average loss: 1.9729, Accuracy: 4819/10000 (48%)

Train Epoch: 76	Loss: 0.656302 Accuracy: 76.89%

      Test set: Average loss: 1.9973, Accuracy: 4837/10000 (48%)

Train Epoch: 77	Loss: 0.649822 Accuracy: 76.92%

      Test set: Average loss: 2.0193, Accuracy: 4858/10000 (49%)

Train Epoch: 78	Loss: 0.647525 Accuracy: 77.09%

      Test set: Average loss: 2

In [19]:
##### 2.3 effect of dropout

dropout = 0.2
## data loaders for training and testing;
train_loader, test_loader, train_data, test_data = data_loader(dataset_name)
## load the multi-layered perceptron (MLP) model
mlp_model = Net(one_more_hid=True)

## optimizer used to implement weight update using gradient descent
optimizer = torch.optim.SGD(mlp_model.parameters(), lr=lr)

## train the model
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []

for epoch_idx in range(epoch_number):
    train_loss_ret, train_acc_ret, _, _ = train(mlp_model, train_loader, optimizer, epoch_idx)
    train_loss_list.append(train_loss_ret)
    train_acc_list.append(train_acc_ret)

    test_loss_ret, test_acc_ret, _, _ = test(mlp_model, test_loader, epoch_idx)
    test_loss_list.append(test_loss_ret)
    test_acc_list.append(test_acc_ret)

## plot loss
x_axis = np.arange(epoch_number)
loss_fig = plt.figure()
plt.plot(x_axis, train_loss_list, 'r')
plt.plot(x_axis, test_loss_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(("train", "test"))
os.makedirs('./output2.3', exist_ok=True)
loss_fig.savefig(os.path.join('./output2.3', dataset_name + "_loss.pdf"))
plt.close(loss_fig)

## plot accuracy
acc_fig = plt.figure()
plt.plot(x_axis, train_acc_list, 'r')
plt.plot(x_axis, test_acc_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(("train", "test"))
os.makedirs('./output2.3', exist_ok=True)
acc_fig.savefig(os.path.join('./output2.3', dataset_name + "_accuracy.pdf"))
plt.close(acc_fig)

## save loss and accuracy to the file
loss_acc_np = np.arange(epoch_number).reshape((-1, 1))
loss_acc_np = np.append(loss_acc_np, np.array(train_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(train_acc_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_acc_list).reshape((-1, 1)), axis=1)
os.makedirs('./output2.3', exist_ok=True)
np.savetxt(os.path.join('./output2.3', dataset_name + "_loss_acc.txt"), loss_acc_np)

Files already downloaded and verified
Files already downloaded and verified
Working on cifar10 dataset ... 
len of train_loader dataset:  50000
len of test_loader dataset:  10000
Train Epoch: 0	Loss: 1.982770 Accuracy: 28.84%

      Test set: Average loss: 1.8059, Accuracy: 3572/10000 (36%)

Train Epoch: 1	Loss: 1.761765 Accuracy: 37.37%

      Test set: Average loss: 1.7253, Accuracy: 3864/10000 (39%)

Train Epoch: 2	Loss: 1.696573 Accuracy: 39.66%

      Test set: Average loss: 1.6700, Accuracy: 4101/10000 (41%)

Train Epoch: 3	Loss: 1.646376 Accuracy: 41.56%

      Test set: Average loss: 1.6341, Accuracy: 4267/10000 (43%)

Train Epoch: 4	Loss: 1.607065 Accuracy: 42.89%

      Test set: Average loss: 1.6090, Accuracy: 4309/10000 (43%)

Train Epoch: 5	Loss: 1.573631 Accuracy: 44.22%

      Test set: Average loss: 1.5996, Accuracy: 4367/10000 (44%)

Train Epoch: 6	Loss: 1.549448 Accuracy: 45.27%

      Test set: Average loss: 1.5811, Accuracy: 4405/10000 (44%)

Train Epoch: 7	Loss: 1.

Train Epoch: 70	Loss: 1.139174 Accuracy: 59.24%

      Test set: Average loss: 1.5496, Accuracy: 4796/10000 (48%)

Train Epoch: 71	Loss: 1.133864 Accuracy: 59.60%

      Test set: Average loss: 1.5552, Accuracy: 4751/10000 (48%)

Train Epoch: 72	Loss: 1.133288 Accuracy: 59.44%

      Test set: Average loss: 1.5636, Accuracy: 4667/10000 (47%)

Train Epoch: 73	Loss: 1.131366 Accuracy: 59.68%

      Test set: Average loss: 1.5378, Accuracy: 4739/10000 (47%)

Train Epoch: 74	Loss: 1.127689 Accuracy: 59.42%

      Test set: Average loss: 1.5414, Accuracy: 4803/10000 (48%)

Train Epoch: 75	Loss: 1.122690 Accuracy: 59.86%

      Test set: Average loss: 1.5559, Accuracy: 4751/10000 (48%)

Train Epoch: 76	Loss: 1.121278 Accuracy: 59.99%

      Test set: Average loss: 1.5700, Accuracy: 4682/10000 (47%)

Train Epoch: 77	Loss: 1.120867 Accuracy: 59.91%

      Test set: Average loss: 1.5813, Accuracy: 4665/10000 (47%)

Train Epoch: 78	Loss: 1.118453 Accuracy: 60.12%

      Test set: Average loss: 1

In [20]:
##### 2.4 free style architecture of ANN

dropout = 0.2
## data loaders for training and testing;
train_loader, test_loader, train_data, test_data = data_loader(dataset_name)
## load the multi-layered perceptron (MLP) model
mlp_model = Net(freestyle=True)

## optimizer used to implement weight update using gradient descent
optimizer = torch.optim.SGD(mlp_model.parameters(), lr=lr)

## train the model
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []

for epoch_idx in range(epoch_number):
    train_loss_ret, train_acc_ret, _, _ = train(mlp_model, train_loader, optimizer, epoch_idx)
    train_loss_list.append(train_loss_ret)
    train_acc_list.append(train_acc_ret)

    test_loss_ret, test_acc_ret, _, _ = test(mlp_model, test_loader, epoch_idx)
    test_loss_list.append(test_loss_ret)
    test_acc_list.append(test_acc_ret)

## plot loss
x_axis = np.arange(epoch_number)
loss_fig = plt.figure()
plt.plot(x_axis, train_loss_list, 'r')
plt.plot(x_axis, test_loss_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(("train", "test"))
os.makedirs('./output2.4', exist_ok=True)
loss_fig.savefig(os.path.join('./output2.4', dataset_name + "_loss.pdf"))
plt.close(loss_fig)

## plot accuracy
acc_fig = plt.figure()
plt.plot(x_axis, train_acc_list, 'r')
plt.plot(x_axis, test_acc_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(("train", "test"))
os.makedirs('./output2.4', exist_ok=True)
acc_fig.savefig(os.path.join('./output2.4', dataset_name + "_accuracy.pdf"))
plt.close(acc_fig)

## save loss and accuracy to the file
loss_acc_np = np.arange(epoch_number).reshape((-1, 1))
loss_acc_np = np.append(loss_acc_np, np.array(train_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(train_acc_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_acc_list).reshape((-1, 1)), axis=1)
os.makedirs('./output2.4', exist_ok=True)
np.savetxt(os.path.join('./output2.4', dataset_name + "_loss_acc.txt"), loss_acc_np)

Files already downloaded and verified
Files already downloaded and verified
Working on cifar10 dataset ... 
len of train_loader dataset:  50000
len of test_loader dataset:  10000
Train Epoch: 0	Loss: 1.891991 Accuracy: 32.08%

      Test set: Average loss: 1.7320, Accuracy: 3777/10000 (38%)

Train Epoch: 1	Loss: 1.728831 Accuracy: 37.83%

      Test set: Average loss: 1.6626, Accuracy: 4054/10000 (41%)

Train Epoch: 2	Loss: 1.674631 Accuracy: 40.14%

      Test set: Average loss: 1.6266, Accuracy: 4214/10000 (42%)

Train Epoch: 3	Loss: 1.630940 Accuracy: 41.46%

      Test set: Average loss: 1.5911, Accuracy: 4296/10000 (43%)

Train Epoch: 4	Loss: 1.600208 Accuracy: 42.52%

      Test set: Average loss: 1.5731, Accuracy: 4374/10000 (44%)

Train Epoch: 5	Loss: 1.585501 Accuracy: 43.59%

      Test set: Average loss: 1.5598, Accuracy: 4431/10000 (44%)

Train Epoch: 6	Loss: 1.563069 Accuracy: 43.78%

      Test set: Average loss: 1.5535, Accuracy: 4444/10000 (44%)

Train Epoch: 7	Loss: 1.

Train Epoch: 70	Loss: 1.269710 Accuracy: 54.91%

      Test set: Average loss: 1.4765, Accuracy: 4817/10000 (48%)

Train Epoch: 71	Loss: 1.268465 Accuracy: 54.97%

      Test set: Average loss: 1.4835, Accuracy: 4794/10000 (48%)

Train Epoch: 72	Loss: 1.265827 Accuracy: 55.32%

      Test set: Average loss: 1.4822, Accuracy: 4785/10000 (48%)

Train Epoch: 73	Loss: 1.266063 Accuracy: 55.07%

      Test set: Average loss: 1.4827, Accuracy: 4761/10000 (48%)

Train Epoch: 74	Loss: 1.265952 Accuracy: 54.89%

      Test set: Average loss: 1.4802, Accuracy: 4806/10000 (48%)

Train Epoch: 75	Loss: 1.262510 Accuracy: 55.30%

      Test set: Average loss: 1.4745, Accuracy: 4836/10000 (48%)

Train Epoch: 76	Loss: 1.258051 Accuracy: 55.36%

      Test set: Average loss: 1.4932, Accuracy: 4782/10000 (48%)

Train Epoch: 77	Loss: 1.259475 Accuracy: 55.35%

      Test set: Average loss: 1.4733, Accuracy: 4818/10000 (48%)

Train Epoch: 78	Loss: 1.263821 Accuracy: 55.15%

      Test set: Average loss: 1

In [20]:
# 2.5
epoch_number = 100
dropout = 0.2
dataset_name = 'fashion_mnist' # cifar10; fashion_mnist
## data loaders for training and testing;
train_loader, test_loader, train_data, test_data = data_loader("fashion_mnist")

## load the multi-layered perceptron (MLP) model
mlp_model = Net(freestyle=True)

## optimizer used to implement weight update using gradient descent
optimizer = torch.optim.SGD(mlp_model.parameters(), lr=lr)

## train the model
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []

for epoch_idx in range(epoch_number):
    train_loss_ret, train_acc_ret, _, _ = train(mlp_model, train_loader, optimizer, epoch_idx)
    train_loss_list.append(train_loss_ret)
    train_acc_list.append(train_acc_ret)

    test_loss_ret, test_acc_ret, _, _ = test(mlp_model, test_loader, epoch_idx)
    test_loss_list.append(test_loss_ret)
    test_acc_list.append(test_acc_ret)

## plot loss
x_axis = np.arange(epoch_number)
loss_fig = plt.figure()
plt.plot(x_axis, train_loss_list, 'r')
plt.plot(x_axis, test_loss_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(("train", "test"))
os.makedirs('./output2.5', exist_ok=True)
loss_fig.savefig(os.path.join('./output2.5', dataset_name + "_loss.pdf"))
plt.close(loss_fig)

## plot accuracy
acc_fig = plt.figure()
plt.plot(x_axis, train_acc_list, 'r')
plt.plot(x_axis, test_acc_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(("train", "test"))
os.makedirs('./output2.5', exist_ok=True)
acc_fig.savefig(os.path.join('./output2.5', dataset_name + "_accuracy.pdf"))
plt.close(acc_fig)

## save loss and accuracy to the file
loss_acc_np = np.arange(epoch_number).reshape((-1, 1))
loss_acc_np = np.append(loss_acc_np, np.array(train_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(train_acc_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_acc_list).reshape((-1, 1)), axis=1)
os.makedirs('./output2.5', exist_ok=True)
np.savetxt(os.path.join('./output2.5', dataset_name + "_loss_acc.txt"), loss_acc_np)

Working on fashion_mnist dataset ... 
len of train_loader dataset:  60000
len of test_loader dataset:  10000
Train Epoch: 0	Loss: 0.795244 Accuracy: 75.14%

      Test set: Average loss: 0.5489, Accuracy: 8137/10000 (81%)

Train Epoch: 1	Loss: 0.554245 Accuracy: 81.25%

      Test set: Average loss: 0.5043, Accuracy: 8275/10000 (83%)

Train Epoch: 2	Loss: 0.505876 Accuracy: 82.65%

      Test set: Average loss: 0.4730, Accuracy: 8354/10000 (84%)

Train Epoch: 3	Loss: 0.477169 Accuracy: 83.60%

      Test set: Average loss: 0.4549, Accuracy: 8403/10000 (84%)

Train Epoch: 4	Loss: 0.455485 Accuracy: 84.22%

      Test set: Average loss: 0.4456, Accuracy: 8446/10000 (84%)

Train Epoch: 5	Loss: 0.442247 Accuracy: 84.48%

      Test set: Average loss: 0.4260, Accuracy: 8500/10000 (85%)

Train Epoch: 6	Loss: 0.428085 Accuracy: 85.05%

      Test set: Average loss: 0.4279, Accuracy: 8498/10000 (85%)

Train Epoch: 7	Loss: 0.418911 Accuracy: 85.40%

      Test set: Average loss: 0.4275, Accurac


      Test set: Average loss: 0.3833, Accuracy: 8705/10000 (87%)

Train Epoch: 71	Loss: 0.274678 Accuracy: 89.86%

      Test set: Average loss: 0.3816, Accuracy: 8714/10000 (87%)

Train Epoch: 72	Loss: 0.275983 Accuracy: 89.99%

      Test set: Average loss: 0.3784, Accuracy: 8677/10000 (87%)

Train Epoch: 73	Loss: 0.272945 Accuracy: 89.98%

      Test set: Average loss: 0.3814, Accuracy: 8698/10000 (87%)

Train Epoch: 74	Loss: 0.271845 Accuracy: 90.01%

      Test set: Average loss: 0.3811, Accuracy: 8714/10000 (87%)

Train Epoch: 75	Loss: 0.270723 Accuracy: 90.15%

      Test set: Average loss: 0.3719, Accuracy: 8744/10000 (87%)

Train Epoch: 76	Loss: 0.271435 Accuracy: 90.17%

      Test set: Average loss: 0.3725, Accuracy: 8709/10000 (87%)

Train Epoch: 77	Loss: 0.263966 Accuracy: 90.36%

      Test set: Average loss: 0.3846, Accuracy: 8673/10000 (87%)

Train Epoch: 78	Loss: 0.270289 Accuracy: 90.11%

      Test set: Average loss: 0.3718, Accuracy: 8703/10000 (87%)

Train Epoch: 

In [27]:
# 2.6
epoch_number = 100
dropout = 0.0
visual_flag = True

## data loaders for training and testing;
train_loader, test_loader, train_data, test_data = data_loader("fashion_mnist")

## load the multi-layered perceptron (MLP) model
mlp_model = Net(vanilla=True)

## optimizer used to implement weight update using gradient descent
optimizer = torch.optim.SGD(mlp_model.parameters(), lr=lr)

## train the model
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []

for epoch_idx in range(epoch_number):
    train_loss_ret, train_acc_ret, _, _ = train(mlp_model, train_loader, optimizer, epoch_idx)
    train_loss_list.append(train_loss_ret)
    train_acc_list.append(train_acc_ret)

    test_loss_ret, test_acc_ret, _, _ = test(mlp_model, test_loader, epoch_idx)
    test_loss_list.append(test_loss_ret)
    test_acc_list.append(test_acc_ret)

## plot loss
x_axis = np.arange(epoch_number)
loss_fig = plt.figure()
plt.plot(x_axis, train_loss_list, 'r')
plt.plot(x_axis, test_loss_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(("train", "test"))
os.makedirs('./output2.6', exist_ok=True)
loss_fig.savefig(os.path.join('./output2.6', dataset_name + "_loss.pdf"))
plt.close(loss_fig)

## plot accuracy
acc_fig = plt.figure()
plt.plot(x_axis, train_acc_list, 'r')
plt.plot(x_axis, test_acc_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(("train", "test"))
os.makedirs('./output2.6', exist_ok=True)
acc_fig.savefig(os.path.join('./output2.6', dataset_name + "_accuracy.pdf"))
plt.close(acc_fig)

## save loss and accuracy to the file
loss_acc_np = np.arange(epoch_number).reshape((-1, 1))
loss_acc_np = np.append(loss_acc_np, np.array(train_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(train_acc_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_acc_list).reshape((-1, 1)), axis=1)
os.makedirs('./output2.6', exist_ok=True)
np.savetxt(os.path.join('./output2.6', dataset_name + "_loss_acc.txt"), loss_acc_np)

Working on fashion_mnist dataset ... 
len of train_loader dataset:  60000
len of test_loader dataset:  10000
Train Epoch: 0	Loss: 0.605402 Accuracy: 79.57%

      Test set: Average loss: 0.5256, Accuracy: 8146/10000 (81%)

Train Epoch: 1	Loss: 0.482038 Accuracy: 83.36%

      Test set: Average loss: 0.4926, Accuracy: 8279/10000 (83%)

Train Epoch: 2	Loss: 0.457500 Accuracy: 84.31%

      Test set: Average loss: 0.4833, Accuracy: 8290/10000 (83%)

Train Epoch: 3	Loss: 0.443046 Accuracy: 84.77%

      Test set: Average loss: 0.4718, Accuracy: 8348/10000 (83%)

Train Epoch: 4	Loss: 0.434478 Accuracy: 85.02%

      Test set: Average loss: 0.4707, Accuracy: 8364/10000 (84%)

Train Epoch: 5	Loss: 0.427836 Accuracy: 85.21%

      Test set: Average loss: 0.4695, Accuracy: 8375/10000 (84%)

Train Epoch: 6	Loss: 0.422503 Accuracy: 85.37%

      Test set: Average loss: 0.4550, Accuracy: 8381/10000 (84%)

Train Epoch: 7	Loss: 0.418337 Accuracy: 85.60%

      Test set: Average loss: 0.4583, Accurac


      Test set: Average loss: 0.4421, Accuracy: 8415/10000 (84%)

Train Epoch: 71	Loss: 0.369562 Accuracy: 87.10%

      Test set: Average loss: 0.4401, Accuracy: 8436/10000 (84%)

Train Epoch: 72	Loss: 0.369344 Accuracy: 87.14%

      Test set: Average loss: 0.4444, Accuracy: 8435/10000 (84%)

Train Epoch: 73	Loss: 0.369210 Accuracy: 87.13%

      Test set: Average loss: 0.4397, Accuracy: 8434/10000 (84%)

Train Epoch: 74	Loss: 0.368116 Accuracy: 87.17%

      Test set: Average loss: 0.4433, Accuracy: 8450/10000 (84%)

Train Epoch: 75	Loss: 0.368566 Accuracy: 87.18%

      Test set: Average loss: 0.4431, Accuracy: 8435/10000 (84%)

Train Epoch: 76	Loss: 0.368391 Accuracy: 87.15%

      Test set: Average loss: 0.4452, Accuracy: 8392/10000 (84%)

Train Epoch: 77	Loss: 0.368274 Accuracy: 87.01%

      Test set: Average loss: 0.4406, Accuracy: 8462/10000 (85%)

Train Epoch: 78	Loss: 0.368002 Accuracy: 87.23%

      Test set: Average loss: 0.4415, Accuracy: 8415/10000 (84%)

Train Epoch: 

In [32]:
from sklearn.metrics import roc_curve

In [90]:
# 2.7.1
import scikitplot as skplt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

epoch_number = 100
dropout = 0.0
visual_flag = False
class_number_fashion_mnist = 2 # number of classes in fashion mnist

## data loaders for training and testing;
train_loader, test_loader, train_data, test_data = data_loader("fashion_mnist")

## load the multi-layered perceptron (MLP) model
mlp_model = Net()

## optimizer used to implement weight update using gradient descent
optimizer = torch.optim.SGD(mlp_model.parameters(), lr=lr)

## train the model
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []
roc_train_label_list = np.array([])
roc_train_pred_list = np.array([])
roc_test_label_list = np.array([])
roc_test_pred_list = np.array([])

for epoch_idx in range(epoch_number):
    train_loss_ret, train_acc_ret, train_epoch_labels, train_epoch_preds = train(mlp_model, train_loader, optimizer, epoch_idx)
    train_loss_list.append(train_loss_ret)
    train_acc_list.append(train_acc_ret)

    test_loss_ret, test_acc_ret, test_epoch_labels, test_epoch_preds = test(mlp_model, test_loader, epoch_idx)
    test_loss_list.append(test_loss_ret)
    test_acc_list.append(test_acc_ret)
    
    if epoch_idx==epoch_number-1:
        roc_train_label_list = np.append(roc_train_label_list, train_epoch_labels.data.numpy())
        roc_train_pred_list = np.append(roc_train_pred_list, train_epoch_preds.data.numpy())
        roc_test_label_list = np.append(roc_test_label_list, test_epoch_labels.data.numpy())
        roc_test_pred_list = np.append(roc_test_pred_list, test_epoch_preds.data.numpy())

## plot loss
x_axis = np.arange(epoch_number)
loss_fig = plt.figure()
plt.plot(x_axis, train_loss_list, 'r')
plt.plot(x_axis, test_loss_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(("train", "test"))
os.makedirs('./output2.7.1', exist_ok=True)
loss_fig.savefig(os.path.join('./output2.7.1', dataset_name + "_loss.pdf"))
plt.close(loss_fig)

## plot accuracy
acc_fig = plt.figure()
plt.plot(x_axis, train_acc_list, 'r')
plt.plot(x_axis, test_acc_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(("train", "test"))
os.makedirs('./output2.7.1', exist_ok=True)
acc_fig.savefig(os.path.join('./output2.7.1', dataset_name + "_accuracy.pdf"))
plt.close(acc_fig)

## plot ROC curve and ROC area for each class
roc_train_label_list = np.concatenate(((1-roc_train_label_list).reshape(-1, 1), roc_train_label_list.reshape(-1, 1)), axis=1)
roc_train_pred_list = np.concatenate(((1-roc_train_pred_list).reshape(-1, 1), roc_train_pred_list.reshape(-1, 1)), axis=1)
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(roc_train_label_list[:, i], roc_train_pred_list[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(roc_train_label_list.ravel(), roc_train_pred_list.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

roc_fig = plt.figure()
lw = 2
plt.plot(fpr['micro'], tpr['micro'], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc['micro'])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
roc_fig.savefig(os.path.join('./output2.7.1', dataset_name + "_roc_curve.pdf"))
plt.close(roc_fig)

## save loss and accuracy to the file
loss_acc_np = np.arange(epoch_number).reshape((-1, 1))
loss_acc_np = np.append(loss_acc_np, np.array(train_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(train_acc_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_acc_list).reshape((-1, 1)), axis=1)
os.makedirs('./output2.7.1', exist_ok=True)
np.savetxt(os.path.join('./output2.7.1', dataset_name + "_loss_acc.txt"), loss_acc_np)

subset_data_list len:  12000
subset_data_list len:  2000
Working on fashion_mnist dataset ... 
len of train_loader dataset:  12000
len of test_loader dataset:  2000
Train Epoch: 0	Loss: 0.116778 Accuracy: 96.80%

      Test set: Average loss: 0.0630, Accuracy: 1955/2000 (98%)

Train Epoch: 1	Loss: 0.050569 Accuracy: 98.38%

      Test set: Average loss: 0.0534, Accuracy: 1962/2000 (98%)

Train Epoch: 2	Loss: 0.043241 Accuracy: 98.58%

      Test set: Average loss: 0.0488, Accuracy: 1967/2000 (98%)

Train Epoch: 3	Loss: 0.039685 Accuracy: 98.72%

      Test set: Average loss: 0.0434, Accuracy: 1972/2000 (99%)

Train Epoch: 4	Loss: 0.037128 Accuracy: 98.76%

      Test set: Average loss: 0.0432, Accuracy: 1970/2000 (98%)

Train Epoch: 5	Loss: 0.035354 Accuracy: 98.91%

      Test set: Average loss: 0.0411, Accuracy: 1971/2000 (99%)

Train Epoch: 6	Loss: 0.034230 Accuracy: 98.83%

      Test set: Average loss: 0.0397, Accuracy: 1971/2000 (99%)

Train Epoch: 7	Loss: 0.032897 Accuracy: 98.9

Train Epoch: 71	Loss: 0.006883 Accuracy: 99.86%

      Test set: Average loss: 0.0324, Accuracy: 1973/2000 (99%)

Train Epoch: 72	Loss: 0.006896 Accuracy: 99.82%

      Test set: Average loss: 0.0256, Accuracy: 1984/2000 (99%)

Train Epoch: 73	Loss: 0.006517 Accuracy: 99.84%

      Test set: Average loss: 0.0269, Accuracy: 1981/2000 (99%)

Train Epoch: 74	Loss: 0.006693 Accuracy: 99.82%

      Test set: Average loss: 0.0253, Accuracy: 1984/2000 (99%)

Train Epoch: 75	Loss: 0.006602 Accuracy: 99.86%

      Test set: Average loss: 0.0268, Accuracy: 1981/2000 (99%)

Train Epoch: 76	Loss: 0.006283 Accuracy: 99.85%

      Test set: Average loss: 0.0286, Accuracy: 1980/2000 (99%)

Train Epoch: 77	Loss: 0.006173 Accuracy: 99.82%

      Test set: Average loss: 0.0249, Accuracy: 1985/2000 (99%)

Train Epoch: 78	Loss: 0.006083 Accuracy: 99.87%

      Test set: Average loss: 0.0249, Accuracy: 1985/2000 (99%)

Train Epoch: 79	Loss: 0.006225 Accuracy: 99.87%

      Test set: Average loss: 0.0276, A

In [148]:
# 2.7.2
import scikitplot as skplt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

epoch_number = 100
dropout = 0.0
visual_flag = False
class_number_fashion_mnist = 2 # number of classes in fashion mnist

## data loaders for training and testing;
train_loader, test_loader, train_data, test_data = data_loader("fashion_mnist")

## load the multi-layered perceptron (MLP) model
mlp_model = Net(logistic=True)

## optimizer used to implement weight update using gradient descent
optimizer = torch.optim.SGD(mlp_model.parameters(), lr=lr)

## train the model
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []
roc_train_label_list = np.array([])
roc_train_pred_list = np.array([])
roc_test_label_list = np.array([])
roc_test_pred_list = np.array([])

for epoch_idx in range(epoch_number):
    train_loss_ret, train_acc_ret, train_epoch_labels, train_epoch_preds = train(mlp_model, train_loader, optimizer, epoch_idx, logit=True)
    train_loss_list.append(train_loss_ret)
    train_acc_list.append(train_acc_ret)

    test_loss_ret, test_acc_ret, test_epoch_labels, test_epoch_preds = test(mlp_model, test_loader, epoch_idx, logit=True)
    test_loss_list.append(test_loss_ret)
    test_acc_list.append(test_acc_ret)
    
    if epoch_idx==epoch_number-1:
        roc_train_label_list = np.append(roc_train_label_list, train_epoch_labels.data.numpy())
        roc_train_pred_list = np.append(roc_train_pred_list, train_epoch_preds.data.numpy())
        roc_test_label_list = np.append(roc_test_label_list, test_epoch_labels.data.numpy())
        roc_test_pred_list = np.append(roc_test_pred_list, test_epoch_preds.data.numpy())

## plot loss
x_axis = np.arange(epoch_number)
loss_fig = plt.figure()
plt.plot(x_axis, train_loss_list, 'r')
plt.plot(x_axis, test_loss_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(("train", "test"))
os.makedirs('./output2.7.2', exist_ok=True)
loss_fig.savefig(os.path.join('./output2.7.2', dataset_name + "_loss.pdf"))
plt.close(loss_fig)

## plot accuracy
acc_fig = plt.figure()
plt.plot(x_axis, train_acc_list, 'r')
plt.plot(x_axis, test_acc_list, 'b')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(("train", "test"))
os.makedirs('./output2.7.2', exist_ok=True)
acc_fig.savefig(os.path.join('./output2.7.2', dataset_name + "_accuracy.pdf"))
plt.close(acc_fig)

## plot ROC curve and ROC area for each class
roc_train_label_list = np.concatenate(((1-roc_train_label_list).reshape(-1, 1), roc_train_label_list.reshape(-1, 1)), axis=1)
roc_train_pred_list = np.concatenate(((1-roc_train_pred_list).reshape(-1, 1), roc_train_pred_list.reshape(-1, 1)), axis=1)
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(roc_train_label_list[:, i], roc_train_pred_list[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(roc_train_label_list.ravel(), roc_train_pred_list.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

roc_fig = plt.figure()
lw = 2
plt.plot(fpr['micro'], tpr['micro'], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc['micro'])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
roc_fig.savefig(os.path.join('./output2.7.2', dataset_name + "_roc_curve.pdf"))
plt.close(roc_fig)

## save loss and accuracy to the file
loss_acc_np = np.arange(epoch_number).reshape((-1, 1))
loss_acc_np = np.append(loss_acc_np, np.array(train_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_loss_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(train_acc_list).reshape((-1, 1)), axis=1)
loss_acc_np = np.append(loss_acc_np, np.array(test_acc_list).reshape((-1, 1)), axis=1)
os.makedirs('./output2.7.2', exist_ok=True)
np.savetxt(os.path.join('./output2.7.2', dataset_name + "_loss_acc.txt"), loss_acc_np)

subset_data_list len:  12000
subset_data_list len:  2000
Working on fashion_mnist dataset ... 
len of train_loader dataset:  12000
len of test_loader dataset:  2000
tensor(11587) 12000
Train Epoch: 0	Loss: 0.116929 Accuracy: 96.56%

      Test set: Average loss: 0.0761, Accuracy: 1947/2000 (97%)

tensor(11786) 12000
Train Epoch: 1	Loss: 0.062092 Accuracy: 98.22%

      Test set: Average loss: 0.0634, Accuracy: 1954/2000 (98%)

tensor(11814) 12000
Train Epoch: 2	Loss: 0.053454 Accuracy: 98.45%

      Test set: Average loss: 0.0579, Accuracy: 1963/2000 (98%)

tensor(11819) 12000
Train Epoch: 3	Loss: 0.049218 Accuracy: 98.49%

      Test set: Average loss: 0.0540, Accuracy: 1964/2000 (98%)

tensor(11826) 12000
Train Epoch: 4	Loss: 0.046517 Accuracy: 98.55%

      Test set: Average loss: 0.0516, Accuracy: 1966/2000 (98%)

tensor(11831) 12000
Train Epoch: 5	Loss: 0.044593 Accuracy: 98.59%

      Test set: Average loss: 0.0502, Accuracy: 1967/2000 (98%)

tensor(11837) 12000
Train Epoch: 6	Lo

tensor(11895) 12000
Train Epoch: 60	Loss: 0.027406 Accuracy: 99.12%

      Test set: Average loss: 0.0353, Accuracy: 1973/2000 (99%)

tensor(11899) 12000
Train Epoch: 61	Loss: 0.027341 Accuracy: 99.16%

      Test set: Average loss: 0.0357, Accuracy: 1973/2000 (99%)

tensor(11900) 12000
Train Epoch: 62	Loss: 0.027362 Accuracy: 99.17%

      Test set: Average loss: 0.0359, Accuracy: 1973/2000 (99%)

tensor(11900) 12000
Train Epoch: 63	Loss: 0.027183 Accuracy: 99.17%

      Test set: Average loss: 0.0353, Accuracy: 1974/2000 (99%)

tensor(11897) 12000
Train Epoch: 64	Loss: 0.027132 Accuracy: 99.14%

      Test set: Average loss: 0.0353, Accuracy: 1974/2000 (99%)

tensor(11894) 12000
Train Epoch: 65	Loss: 0.026808 Accuracy: 99.12%

      Test set: Average loss: 0.0358, Accuracy: 1972/2000 (99%)

tensor(11900) 12000
Train Epoch: 66	Loss: 0.026740 Accuracy: 99.17%

      Test set: Average loss: 0.0353, Accuracy: 1972/2000 (99%)

tensor(11899) 12000
Train Epoch: 67	Loss: 0.026808 Accuracy: 9